จาก ep ที่แล้ว ที่เราพัฒนาโปรแกรม [AI การแพทย์ วินิจฉัยภาวะปอดรั่ว Pneumothorax](https://www.bualabs.com/archives/2856/ai-diagnose-pneumothorax-read-chest-x-ray-radiograph-image-collapsed-lung-radiologist-triage-patient-classification-segment-medical-image-segmentation-ep-2/) ได้ผลดีพอสมควร ใน ep นี้เราจะปรับปรุงโมเดล ด้วยเทคนิคใหม่ ๆ เช่น [fastai2](https://www.bualabs.com/archives/4102/tutorial-fastai2-oxford-pets-dog-cat-37-class-machine-learning-deep-neural-networks-image-classification-ep-7/), [Data Echoing](https://www.bualabs.com/archives/4157/data-echoing-faster-neural-network-training-with-data-echoing-machine-learning-data-echoing-preprocessing-ep-6/), [Focal Loss](https://www.bualabs.com/archives/4227/what-is-focal-loss-loss-function-ep-4/), และ Ranger Optimizer แบบใหม่ ว่าจะได้ผลเป็นอย่างไร

# 0. Install

In [ ]:
! nvidia-smi

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

เราจะต้อง Install kaggle เพื่อ Download Dataset

In [ ]:
# ## Colab
# ! pip install fastai2 pydicom kornia kaggle -q

In [ ]:
# ## Colab

# !pip install torch-geometric \
#   torch-sparse==latest+cu101 \
#   torch-scatter==latest+cu101 \
#   torch-cluster==latest+cu101 \
#   -f https://pytorch-geometric.com/whl/torch-1.4.0.html 

# 1. Import Library

Import pydicom สำหรับเปิดไฟล์ภาพเอ็กซ์เรย์ DICOM นามสกุล dcm และ Libray อื่น ๆ ที่เราต้องการใช้

In [ ]:
import math
import numpy as np
import pandas as pd

import os
import gc
import glob
import PIL 

import pydicom

from pathlib import Path
from matplotlib import cm
from matplotlib import pyplot as plt

from fastai2.basics import *
from fastai2.vision.all import *
from fastai2.callback.all import *
from fastai2.metrics import *
from fastai2.medical.imaging import *

from IPython.core.debugger import set_trace

import kornia

In [ ]:
set_seed(1234)

# 2. เตรียม Path สำหรับดาวน์โหลดข้อมูล

กำหนด path ของ Config File และ Dataset ว่าจะอยู่ใน Google Drive ถ้าเราใช้ [Google Colab](https://www.bualabs.com/archives/1687/what-is-colab-open-jupyter-notebook-in-github-on-google-colab-create-open-in-colab-button-colab-ep-1/) หรือ อยู่ใน HOME ถ้าเราใช้ VM ธรรมดา และกำหนด Environment Variable ไปยังโฟลเดอร์ที่เก็บ kaggle.json

ในกรณีใช้ Colab ให้ Mount Google Drive เพื่อดึง Config File มาจาก Google Drive ส่วนตัวของเรา เมื่อเรารัน Cell ด้านล่างจะมีลิงค์ปรากฎขึ้นมาให้เรา Login กด Approve แล้ว Copy Authorization Code มาใส่ในช่องด้านล่าง แล้วกด Enter

ในเคสนี้เราจะดึงข้อมูลจากหลาย Dataset เนื่องจากเราจะใช้การเทรนแบบ Progressive Resizing ใช้รูปหลายขนาดในการเทรน 

In [ ]:
dataset = 'siim-acr-pneumothorax-segmentation'
dataset2 = 'jesperdramsch/siim-acr-pneumothorax-segmentation-data'
# dataset3 = 'iafoss/siimacr-pneumothorax-segmentation-data-128'
# dataset4 = 'iafoss/siimacr-pneumothorax-segmentation-data-256'
# dataset5 = 'iafoss/siimacr-pneumothorax-segmentation-data-512'
dataset6 = 'iafoss/siimacr-pneumothorax-segmentation-data-1024'
dataset_test = 'ivanzhovannik/siim_stage2_png'

# Google Colab
config_path = Path('/content/drive')
learner_path = config_path/"My Drive"
data_path_base = Path('/content/datasets/')

data_path = data_path_base/dataset
data_path2 = data_path_base/dataset2
# data_path3 = data_path_base/dataset3
# data_path4 = data_path_base/dataset4
# data_path5 = data_path_base/dataset5
data_path6 = data_path_base/dataset6
data_path_test = data_path_base/dataset_test

from google.colab import drive
drive.mount(str(config_path))
os.environ['KAGGLE_CONFIG_DIR'] = f"{config_path}/My Drive/.kaggle"

# # VM
# config_path = Path(os.getenv("HOME"))
# learner_path = config_path
# data_path = config_path/"datasets"/dataset
# data_path2 = config_path/"datasets"/dataset2
# data_path3 = config_path/"datasets"/dataset3
# data_path4 = config_path/"datasets"/dataset4
# data_path5 = config_path/"datasets"/dataset5
# data_path6 = config_path/"datasets"/dataset6
# data_path_test = config_path/"datasets"/dataset_test

# data_path.mkdir(parents=True, exist_ok=True)
# os.environ['KAGGLE_CONFIG_DIR'] = f"{config_path}/.kaggle"

# 3. Dataset

ในเคสนี้ เราจะ Download ข้อมูล Dataset ที่เกี่ยวข้องทั้งหมดมาเก็บไว้ แบ่งเป็นรูปฟิล์ม X-Ray ขนาดต่าง ๆ ข้อมูลต้นฉบับ ข้อมูล Test Set, etc.

Dataset เราจะดึงจาก Kaggle วิธี Download kaggle.json ให้ดูจาก ep ที่แล้ว

เมื่อได้ kaggle.json มาแล้ว ในกรณีใช้ Google Colab ให้นำมาใส่ไว้ในโฟลเดอร์ My Drive/.kaggle ใน Google Drive ของเรา เป็น My Drive/.kaggle/kaggle.json ถ้าใช้ VM ให้ใส่ใน HOME/.kaggle/

สั่งดาวน์โหลด Dataset จาก Kaggle พร้อมทั้ง unzip ไว้ใน data_path

In [ ]:
# !kaggle competitions download -c {dataset} -p "{data_path}"
# !kaggle datasets download {dataset2} -p "{data_path2}" --unzip
# ## !kaggle datasets download {dataset3} -p "{data_path3}" --unzip
# ## !kaggle datasets download {dataset4} -p "{data_path4}" --unzip
# ## !kaggle datasets download {dataset5} -p "{data_path5}" --unzip
# !kaggle datasets download {dataset6} -p "{data_path6}" --unzip
# !kaggle datasets download {dataset_test} -p "{data_path_test}" --unzip

Unzip ไฟล์ที่ดาวน์โหลดจาก Kaggle Competition

In [ ]:
# ! unzip -q {data_path}/siim-acr-pneumothorax-segmentation.zip -d {data_path}
# ! unzip -q {data_path}/stage_2_images.zip -d {data_path}/stage_2_images
# ! unzip -q {data_path}/stage_2_train.csv.zip -d {data_path}

Import ฟังก์ชันสำหรับแปลงข้อมูล Label ที่อยู่ในรูปแบบ Run-length encoding (RLE) to Mask ที่ให้มากับ Dataset

In [ ]:
import sys
sys.path.insert(0, str(data_path))

from mask_functions import *

# 4. Data

## 4.1 DICOM Image

เปิดดูข้อมูลฟิล์มเอ็กซ์เรย์

In [ ]:
def show_dcm_info(dataset):
    print("Filename.........:", file_path)
    print("Storage type.....:", dataset.SOPClassUID)
    print()

    pat_name = dataset.PatientName
    display_name = pat_name.family_name + ", " + pat_name.given_name
    print("Patient's name......:", display_name)
    print("Patient id..........:", dataset.PatientID)
    print("Patient's Age.......:", dataset.PatientAge)
    print("Patient's Sex.......:", dataset.PatientSex)
    print("Modality............:", dataset.Modality)
    print("Body Part Examined..:", dataset.BodyPartExamined)
    print("View Position.......:", dataset.ViewPosition)
    
    if 'PixelData' in dataset:
        rows = int(dataset.Rows)
        cols = int(dataset.Columns)
        print("Image size.......: {rows:d} x {cols:d}, {size:d} bytes".format(
            rows=rows, cols=cols, size=len(dataset.PixelData)))
        if 'PixelSpacing' in dataset:
            print("Pixel spacing....:", dataset.PixelSpacing)

def plot_pixel_array(dataset, figsize=(10,10)):
    plt.figure(figsize=figsize)
    plt.imshow(dataset.pixel_array, cmap=plt.cm.bone)
    plt.show()


วนลูปเปิดดูไฟล์ DICOM ที่จะมีทั้งรูป และ Metadata ข้อมูลประกอบ อยู่ในไฟล์เดียวกัน

In [ ]:
for file_path in glob.glob(str(data_path2/'pneumothorax/dicom-images-train/*/*/*.dcm')):
    dataset = pydicom.dcmread(file_path)
    show_dcm_info(dataset)
    plot_pixel_array(dataset)
    break # Comment this out to see all

ลองดูทีละหลาย ๆ ไฟล์

In [ ]:
start = 33   # Starting index of images
num_img = 3 # Total number of images to show

fig, ax = plt.subplots(nrows=1, ncols=num_img, sharey=True, figsize=(num_img*5,5))
for q, file_path in enumerate(glob.glob(str(data_path2/'pneumothorax/dicom-images-train/*/*/*.dcm'))[start:start+num_img]):
    dataset = pydicom.dcmread(file_path)
    #show_dcm_info(dataset)
    
    ax[q].imshow(dataset.pixel_array, cmap=plt.cm.bone)


ดู Label ที่มาในรูปแบบ Run-length encoding (RLE) ดังที่เห็นใน Column EncodedPixels 

-1 หมายถึง ปอดปกติ

In [ ]:
df = pd.read_csv(data_path2/'train-rle.csv', index_col=0)
df.head()

In [ ]:
df.shape

In [ ]:
samples_per_cls = [len(df[df[' EncodedPixels']==' -1']), len(df[df[' EncodedPixels']!=' -1'])]
samples_per_cls

## 4.2 Mask

ใช้ฟังก์ชันที่ได้มา แปลง RLE เป็น Mask ขนาด 1024 x 1024

In [ ]:
mask = rle2mask(df.loc['1.2.276.0.7230010.3.1.4.8323329.4904.1517875185.355709'][0].strip(), 1024, 1024).T
# a = mask.sum(axis=1)
# a = mask.sum(axis=0)
# a.argmax()

mask[105:115, 300:310]

เพื่อให้เห็นชัด ๆ เราจะแปลงเป็น Transparent ไว้สำหรับ Overlay กับภาพฟิล์ม X-Ray

In [ ]:
mask_rgba = mask/255
mask_rgba = np.repeat(mask_rgba[:, :, np.newaxis], 4, axis=2)
mask_rgba[:, :, [1,2]] = 0
mask_rgba[:, :, 3] = mask_rgba[:, :, 3]*0.7

mask_rgba[105:115, 300:310, 0]

ลองแสดง Mask เป็นรูป

In [ ]:
plt.imshow(mask_rgba)

## 4.3 DICOM Image and Mask

แสดงฟิล์ม X-Ray พร้อมระบายสี Pneumothorax ด้วย Mask สีแดง

In [ ]:
start = 22   # Starting index of images
num_img = 3 # Total number of images to show

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=num_img, sharey=True, figsize=(num_img*5,5))
for q, file_path in enumerate(glob.glob(str(data_path2/'pneumothorax/dicom-images-train/*/*/*.dcm'))[start:start+num_img]):
    dataset = pydicom.dcmread(file_path)
    print(file_path.split('/')[-1])
    ax[q].imshow(dataset.pixel_array, cmap=plt.cm.bone)
    fn = file_path.split('/')[-1][:-4]
    rle = df.loc[fn][0].strip()
    # print(rle)
    if rle != '-1':
        print('See Marker')
        mask = rle2mask(rle, 1024, 1024).T
        ax[q].set_title('See Marker')
#         # Normal Mask
#         ax[q].imshow(mask, alpha=0.3, cmap="rainbow")
        
        # Make alpha mask
        mask_rgba = mask/255
        mask_rgba = np.repeat(mask_rgba[:, :, np.newaxis], 4, axis=2)
        mask_rgba[:, :, [1,2]] = 0
        mask_rgba[:, :, 3] = mask_rgba[:, :, 3]*0.4

        ax[q].imshow(mask_rgba)
    else:
        print('Nothing to see')
        ax[q].set_title('Nothing to see')


# 5. Data Pipeline

เมื่อเราได้รูปด้านบนมาแล้ว เพื่อความสำดวก เราจะแปลงทั้งหมดเป็นไฟล์รูป .png เนื่องจาก Lossless แล้ว Resize เป็นขนาดต่าง ๆ ใส่ไว้ใน Folder ที่กำหนด ซึ่งมีคนทำไว้ให้เราแล้ว เราก็ไป Download มาใน Dataset 3, 4, 5, 6

กำหนด [Batch Size](https://www.bualabs.com/archives/729/what-is-batch-size-in-deep-neural-networks-how-to-adjust-machine-learning-model-accuracy-deep-learning-hyperparameter-tuning-ep-2/), ขนาดของรูปที่จะเทรน และ Path ที่เก็บรูป

In [ ]:
bs=64
e=2
SZ = 128
# path = data_path3
path = data_path6

ใช้ [Data Echoing](https://www.bualabs.com/archives/4157/data-echoing-faster-neural-network-training-with-data-echoing-machine-learning-data-echoing-preprocessing-ep-6/)

In [ ]:
class EchoingTransform(ItemTransform):
    order = 2
    split_idx = 0
    def __init__(self, e): self.e = e
    def encodes(self, x):
        img, lbl = x
        if self.e > 1:
            img = img.repeat(self.e, 1, 1, 1)
            lbl = lbl.repeat(self.e, 1, 1)
        # print(img.shape)
        # print(lbl.shape)
        return img, lbl

เนื่องจาก mask อยู่ใน format ที่ไม่ได้เป็น 0, 1 แต่เป็นไล่สีขาวดำ 0, 64, 
128, 191, 255 เราจะต้องปรับให้เป็น class 0, 1

In [ ]:
def label_func(fn): return str(fn).replace('train/train', 'masks')

In [ ]:
fn = get_image_files(path/'train')[111]
msk_fn = label_func(fn)
msk = PILMask.create(msk_fn)
msk

In [ ]:
tensor(msk)[120:130, 420:430]

In [ ]:
def get_msk(fn):

  msk_fn = label_func(fn)
  msk = np.array(PILMask.create(msk_fn))
  
  msk[msk!=0] = 1
  return PILMask.create(msk)

ลองดู Mask หลังจากแปลงเรียบร้อยแล้ว

In [ ]:
msk2 = get_msk(fn)
tensor(msk2)[120:130, 420:430]

In [ ]:
get_y = lambda o: get_msk(o)

ใช้ [DataBlock API](https://www.bualabs.com/archives/2693/data-block-api-data-pipeline-machine-learning-supervised-learning-preprocessing-ep-5/) สร้าง [DataBunch](https://www.bualabs.com/archives/2318/databunch-learner-refactor-neural-network-training-loop-neural-network-ep-9/) เตรียมส่งให้ Model สำหรับเทรน โดย [Train/Validation Split](https://www.bualabs.com/archives/532/what-is-training-set-why-train-test-split-training-set-validation-set-test-set/) 80/20, แต่ละ Pixel เป็น Binary Classification 0/1 , ทำ [Data Augmentation](https://www.bualabs.com/archives/1170/what-is-data-augmentation-benefits-data-augmentaion-deep-learning-training-model-deep-neural-network-regularization-ep-1/) ทุกอย่างยกเว้น Flip Vertical แล้ว [Normalize](https://www.bualabs.com/archives/2100/what-is-normalization-feature-scaling-rescaling-normalization-standardization-feedforward-train-machine-learning-preprocessing-ep-2/) ด้วย imagenet_stats

In [ ]:
codes = np.array(["Normal", "Pneumothorax"])

In [ ]:
# aug_transforms??

In [ ]:
def getDataBlock(size, e):
  return DataBlock(blocks=(ImageBlock, MaskBlock(codes)),
                  get_items = get_image_files,
                  get_y = get_y,
                  splitter=RandomSplitter(), 
                  item_tfms=RandomResizedCrop(size, min_scale=0.7, ratio=(1, 1)), 
                  # item_tfms=RandomResizedCrop(size), 
                  # batch_tfms = [EchoingTransform(e), *aug_transforms(flip_vert=True, max_rotate=180.0)])
                  batch_tfms = [EchoingTransform(e), *aug_transforms(do_flip=False)])


In [ ]:
data = getDataBlock(SZ, e)

In [ ]:
# path = data_path6
# data.summary(path/'train')

In [ ]:
dls = data.dataloaders(path/'train', bs=bs)

ได้ Traing Set และ Validation Set จำนวนดังนี้

In [ ]:
len(dls.train_ds), len(dls.valid_ds)

ดูข้อมูล 1 Batch 

In [ ]:
dls.show_batch(max_n=9, vmin=0, vmax=1, cmap='Reds', figsize=(12, 12))

# 6. Model

## 6.1 Dice Metric

Dice หรือ Sørensen–Dice coefficient เป็นการคำนวนสถิติที่ใช้วัดค่าความเหมือนกัน ของ 2 ข้อมูลตัวอย่าง ดังสมการด้านล่าง รายละเอียดเพิ่มเติมใน ep [Metrics / Confusion Matrix](https://www.bualabs.com/archives/1968/what-is-metrics-confusion-matrix-accuracy-precision-recall-f1-score-difference-metrics-ep-1/)

$$DSC =  \frac{2 |X \cap Y|}{|X|+ |Y|} = \frac{2 TP}{2 TP + FP + FN} $$

## 6.2 U-Net and ResNet34

เราจะใช้ [Model Architecture](https://www.bualabs.com/archives/2703/how-to-read-model-convolutional-neural-network-shape-activation-map-model-architecture-convnet-ep-7/) เหมือน ep ที่แล้ว คือ U-Net และ ResNet34 แปลงโมเดลเป็นแบบ Mixed Precision Training และใช้ [Focal Loss](https://www.bualabs.com/archives/4227/what-is-focal-loss-loss-function-ep-4/) แทน Cross Entropy Loss ตามปกติ

In [ ]:
class XFocalLoss(kornia.losses.FocalLoss):
    y_int = True
    def __init__(self, alpha: float, gamma: float = 2.0,
                 reduction: str = 'none') -> None:
        super(XFocalLoss, self).__init__(alpha, gamma, reduction)

    def forward(  # type: ignore
            self,
            input: torch.Tensor,
            target: torch.Tensor) -> torch.Tensor:       

        # set_trace()
        # print(input.shape)
        # print(target.shape)
        return super().forward(input, target)      

    def decodes(self, x):    return x.argmax(dim=1)
    def activation(self, x): return F.softmax(x, dim=1)               

class XDiceLoss(kornia.losses.DiceLoss):
    y_int = True
    def __init__(self) -> None:
        super(XDiceLoss, self).__init__()

    def forward(  # type: ignore
            self,
            input: torch.Tensor,
            target: torch.Tensor) -> torch.Tensor:       

        # set_trace()
        # print(input.shape)
        # print(target.shape)
        return super().forward(input, target)      

    def decodes(self, x):    return x.argmax(dim=1)
    def activation(self, x): return F.softmax(x, dim=1)                   

หาสัดส่วนของ Mask ต่อภาพทั้งหมด

In [ ]:
b = dls.one_batch()
b[1].shape, b[1].float().mean()

In [ ]:
# loss_func = kornia.losses.FocalLoss(alpha= 0.5, gamma=2.0, reduction='mean')

# loss_func = LabelSmoothingCrossEntropy()

# weights = torch.tensor([[0.9]*1 + [1.1]]).cuda()
# loss_func = CrossEntropyLossFlat(weight=weights, axis=1)

# loss_func = CrossEntropyLossFlat(axis=1)

loss_func = XFocalLoss(alpha=100.0, gamma=2.0, reduction='mean')

# loss_func = XDiceLoss()

# beta = 0.99
# no_of_classes = 2
# effective_num = 1.0 - np.power(beta, samples_per_cls)
# weights = (1.0 - beta) / np.array(effective_num)
# weights = weights / np.sum(weights) * no_of_classes
# weights = (1.0 - np.power(beta, samples_per_cls))
# weights = tensor(weights).cuda()
# loss_func = CrossEntropyLossFlat(weight=weights, axis=1)

In [ ]:
# unet_config??

In [ ]:
metrics = [foreground_acc, Dice(), JaccardCoeff()]
config = unet_config(self_attention=True, blur=True, act_cls=Mish)
# config = unet_config(self_attention=True, blur=True)

opt_func = ranger

In [ ]:
# def getLearner(dls):
#     return unet_learner(dls, resnet34, metrics=metrics, config=config,
#                      loss_func=loss_func, 
#                      pretrained=True,                      
#                      path=learner_path, 
#                      opt_func=opt_func, 
#                      wd=1e-1, 
#                      cbs=[ShowGraphCallback]).to_fp16()

arch = partial(xresnet34, pretrained=False, c_in=3, act_cls=Mish, sa=True, n_out=2)

def getLearner(dls):
    return unet_learner(dls, arch, metrics=metrics, config=config,
                     loss_func=loss_func, 
                     pretrained=False,                      
                     path=learner_path, 
                     opt_func=opt_func, 
                     wd=1e-1, 
                     cbs=[ShowGraphCallback]).to_fp16()                     

In [ ]:
learn = getLearner(dls)

In [ ]:
learn.summary()

# 7. Train

เราจะใช้ Progressive Resizing ข้อมูลตัวอย่างที่ป้อนให้กับโมเดล เหมือนใน [Image Segmentation ep.1](https://www.bualabs.com/archives/835/image-segmentation-semantic-segmentation-camvid-machine-learning-unet-deep-learning-image-segmentation-ep-1/) เริ่มต้น จาก 128, 256, 512 ไปจนถึง 1024 ที่เป็นขนาดต้นฉบับ 

## 7.1 Train 128

In [ ]:
# learn.freeze()

In [ ]:
# learn.lr_find()

In [ ]:
# learn.fit_one_cycle(1, lr_max=slice(3e-3), pct_start=0.8)
learn.fit_flat_cos(2, lr_max=slice(1e-2))

In [ ]:
# learn.fit_one_cycle(10, lr_max=slice(3e-3), pct_start=0.8)
learn.fit_flat_cos(6, lr_max=slice(6e-3))

In [ ]:
# learn.fit_one_cycle(10, lr_max=slice(3e-3), pct_start=0.8)
learn.fit_flat_cos(4, lr_max=slice(3e-3))

In [ ]:
# learn.save('03c-unet-resnet34-128-1')

In [ ]:
# learn.load('03c-unet-resnet34-128-1');

In [ ]:
# learn.unfreeze()

In [ ]:
# learn.lr_find()

In [ ]:
# learn.fit_flat_cos(2, lr=slice(1e-6, 1e-4))
# learn.fit_one_cycle(4, lr_max=slice(1e-6, 1e-4), pct_start=0.7)

In [ ]:
learn.save('03c-unet-resnet34-128-2')

In [ ]:
# learn.show_results(max_n=9, vmin=0, vmax=1, cmap='Reds', figsize=(12, 12))

## 7.2 Increase Image size to 256

In [ ]:
learn = None
dls = None
gc.collect()
torch.cuda.empty_cache()

In [ ]:
bs = 32
e = 2
SZ = 224
# path = data_path4
path = data_path6

In [ ]:
data = getDataBlock(SZ, e)

In [ ]:
dls = data.dataloaders(path/'train', bs=bs)

In [ ]:
# dls.show_batch()

In [ ]:
learn = getLearner(dls)

In [ ]:
learn.load('03c-unet-resnet34-128-2')
learn.freeze()

In [ ]:
# learn.lr_find()

In [ ]:
learn.fit_flat_cos(6, lr=slice(3e-4))
# learn.fit_one_cycle(10, lr_max=slice(1e-3), pct_start=0.8)

In [ ]:
learn.fit_flat_cos(4, lr=slice(1e-4))
# learn.fit_one_cycle(10, lr_max=slice(1e-3), pct_start=0.8)

In [ ]:
learn.save('03c-unet-resnet34-256-1')

In [ ]:
# learn.load('03c-unet-resnet34-256-1');

In [ ]:
learn.unfreeze()

In [ ]:
# learn.lr_find()

In [ ]:
learn.fit_flat_cos(4, lr=slice(1e-6, 1e-4))
# learn.fit_one_cycle(4, lr_max=slice(1e-6, 1e-4), pct_start=0.7)

In [ ]:
learn.save('03c-unet-resnet34-256-2')

In [ ]:
# learn.load('03c-unet-resnet34-256-2');

In [ ]:
# learn.show_results(max_n=9, vmin=0, vmax=1, cmap='Reds', figsize=(12, 12))

## 7.3 Increase Image size to 384

In [ ]:
learn = None
dls = None
gc.collect()
torch.cuda.empty_cache()

In [ ]:
bs=14
e=2
SZ = 336
path = data_path6

In [ ]:
data = getDataBlock(SZ, e)

In [ ]:
dls = data.dataloaders(path/'train', bs=bs)

In [ ]:
# dls.show_batch()

In [ ]:
learn = getLearner(dls)

In [ ]:
learn.load('03c-unet-resnet34-256-2')
learn.freeze()

In [ ]:
# learn.lr_find()

In [ ]:
learn.fit_flat_cos(6, lr=slice(1e-4))
# learn.fit_one_cycle(8, lr_max=slice(3e-4), pct_start=0.8)

In [ ]:
learn.fit_flat_cos(4, lr=slice(3e-5))
# learn.fit_one_cycle(8, lr_max=slice(3e-4), pct_start=0.8)

In [ ]:
learn.save('03c-unet-resnet34-384-1')

In [ ]:
# learn.load('03c-unet-resnet34-384-1');

In [ ]:
learn.unfreeze()

In [ ]:
# learn.lr_find()

In [ ]:
learn.fit_flat_cos(4, lr=slice(3e-7, 3e-5))
# learn.fit_one_cycle(2, lr_max=slice(1e-6, 1e-4), pct_start=0.7)

In [ ]:
learn.save('03c-unet-resnet34-384-2')

In [ ]:
# learn.show_results(max_n=9, vmin=0, vmax=1, cmap='Reds', figsize=(12, 12))

## 7.4 Increase Image size to 512 

In [ ]:
learn = None
dls = None
gc.collect()
torch.cuda.empty_cache()

In [ ]:
bs=17
e=1
SZ = 448
path = data_path6

In [ ]:
data = getDataBlock(SZ, e)

In [ ]:
dls = data.dataloaders(path/'train', bs=bs)

In [ ]:
# dls.show_batch()

In [ ]:
learn = getLearner(dls)

In [ ]:
learn.load('03c-unet-resnet34-384-2')
learn.freeze()

In [ ]:
# learn.lr_find()

In [ ]:
learn.fit_flat_cos(4, lr=slice(1e-4))
# learn.fit_one_cycle(8, lr_max=slice(3e-4), pct_start=0.8)

In [ ]:
learn.fit_flat_cos(6, lr=slice(3e-5))
# learn.fit_one_cycle(8, lr_max=slice(3e-4), pct_start=0.8)

In [ ]:
learn.save('03c-unet-resnet34-512-1')

In [ ]:
# learn.load('03c-unet-resnet34-512-1');

In [ ]:
learn.unfreeze()

In [ ]:
# learn.lr_find()

In [ ]:
learn.fit_flat_cos(2, lr=slice(3e-7, 3e-5))
# learn.fit_one_cycle(2, lr_max=slice(3e-7, 3e-5), pct_start=0.7)

In [ ]:
learn.save('03c-unet-resnet34-512-2')

In [ ]:
# learn.show_results(max_n=9, vmin=0, vmax=1, cmap='Reds', figsize=(12, 12))

## 7.5 Data Augmentation Annealing

ก่อน Infer เราจะเทรนด้วยข้อมูลที่ไม่ได้ทำ [Data Augmentation](https://www.bualabs.com/archives/1170/what-is-data-augmentation-benefits-data-augmentaion-deep-learning-training-model-deep-neural-network-regularization-ep-1/) ตบท้าย

In [ ]:
learn = None
dls = None
gc.collect()
torch.cuda.empty_cache()

In [ ]:
bs=17
e=1
SZ = 448
path = data_path6

In [ ]:
# path.ls()

In [ ]:
# Resize??

In [ ]:
data = DataBlock(blocks=(ImageBlock, MaskBlock(codes)),
                  get_items = get_image_files,
                  get_y = get_y,
                  splitter=RandomSplitter(0.01), 
                  item_tfms=Resize(SZ), 
                  # item_tfms=Resize(SZ, method='squish'), 
                #   item_tfms=RandomResizedCrop(SZ, min_scale=0.9, ratio=(1, 1)), 
                  # batch_tfms=[EchoingTransform(e)])
                  )                  

In [ ]:
dls = data.dataloaders(path/'train', bs=bs)

In [ ]:
# dls.show_batch()

In [ ]:
learn = getLearner(dls)

In [ ]:
learn.load('03c-unet-resnet34-512-2');

In [ ]:
learn.unfreeze()

In [ ]:
# learn.lr_find()

In [ ]:
learn.fit_flat_cos(2, lr=slice(3e-7, 3e-5))
# learn.fit_one_cycle(2, lr_max=slice(3e-7, 3e-5), pct_start=0.7)
# learn.fit_one_cycle(4, lr_max=slice(3e-7, 3e-5))

In [ ]:
learn.save('03c-unet-resnet34-512-3')

In [ ]:
# learn.load('03c-unet-resnet34-512-3');

In [ ]:
learn.show_results(max_n=9, vmin=0, vmax=1, cmap='Reds', figsize=(12, 12))

# 8. Make a Prediction

เทรนเสร็จแล้ว เราจะใช้โมเดลมาทำนายวินิจฉัย Pneumothorax หรือ ภาวะปอดรั่ว จากฟิล์ม X-Ray ใน Validation Set ที่เราแบ่งไว้ตอนแรก 20% ว่าจะตรงไหม

In [ ]:
gc.collect()
torch.cuda.empty_cache()

In [ ]:
# Predictions for the val set
preds, ys = learn.get_preds()
preds.shape, ys.shape

In [ ]:
preds = preds[:,1,...]
ys = ys.squeeze()

In [ ]:
preds.shape, ys.shape

In [ ]:
def dice_overall(preds, targs):
    n = preds.shape[0]
    preds = preds.view(n, -1)
    targs = targs.view(n, -1)
    intersect = (preds * targs).sum(-1).float()
    union = (preds+targs).sum(-1).float()
    u0 = union==0
    intersect[u0] = 1
    union[u0] = 2
    return (2. * intersect / union)

หา Threshold ให้ได้ ค่า Dice ที่ดีที่สุด

In [ ]:
# Find optimal threshold
dices = []
thrs = np.arange(0.01, 1, 0.01)
for i in progress_bar(thrs):
    preds_m = (preds>i).long()
    dices.append(dice_overall(preds_m, ys).mean())
dices = np.array(dices)

In [ ]:
best_dice = dices.max()
best_thr = thrs[dices.argmax()]

plt.figure(figsize=(8,4))
plt.plot(thrs, dices)
plt.vlines(x=best_thr, ymin=dices.min(), ymax=dices.max())
plt.text(best_thr+0.03, best_dice-0.01, f'DICE = {best_dice:.3f}', fontsize=14);
plt.show()

ดูผลลัพธ์ในการวิจิฉัยภาวะปอดรั่ว (Pneumothorax) จะเห็นเป็น Image Segmentation เป็นสีเหลืองเขียว บริเวณปอดที่มีภาวะปอดรั่ว (Pneumothorax) เทียบกับ Label จากข้อมูลตัวอย่างใน Dataset

จะเห็นว่าโมเดลของเราวินิจฉัยได้อย่างแม่นยำ 

In [ ]:
# Plot some samples
rows = 5
plot_idx = ys.sum((1,2)).sort(descending=True).indices[:rows]
for idx in plot_idx:
    fig, (ax0, ax1, ax2) = plt.subplots(ncols=3, figsize=(12, 4))
    ax0.imshow(dls.valid_ds[idx][0])
    ax1.imshow(ys[idx], vmin=0, vmax=1)
    ax2.imshow(preds[idx], vmin=0, vmax=1)
    ax1.set_title('Targets')
    ax2.set_title('Predictions')

# 10. Submit Predictions to Kaggle

ส่งผลการวินิจฉัยฟิล์ม X-Ray ใน Test Set ไปให้ Kaggle ตรวจ

In [ ]:
tst_fns = get_image_files(data_path_test)

In [ ]:
dl = learn.dls.test_dl(tst_fns)

In [ ]:
# dl.show_batch()

In [ ]:
b = dl.one_batch()
b[0][1]

In [ ]:
gc.collect()
torch.cuda.empty_cache()

In [ ]:
# Predictions for test set
preds, _ = learn.get_preds(dl=dl)
preds = (preds[:,1,...]>best_thr).long().numpy()
# print(preds.sum())

In [ ]:
preds.shape

In [ ]:
# Plot some samples
rows = 5
plot_idx = np.arange(rows)
for idx in plot_idx:
    fig, (ax0, ax1) = plt.subplots(ncols=2, figsize=(8, 4))
    ax0.imshow(dl.dataset[idx][0])
    ax1.imshow(preds[idx], vmin=0, vmax=1)
    ax1.set_title('Predictions')

In [ ]:
# preds[1, 90:110, 110:130]

In [ ]:
# p = preds[1]
# tst_im = PIL.Image.fromarray((p.T*255).astype(np.uint8)).resize((1024,1024), resample=PIL.Image.NEAREST)
# tst_im = np.asarray(tst_im)
# tst_im[240:260, 200:220]

In [ ]:
# tst_msk = mask2rle(tst_im, 1024, 1024)
# tst_msk

แปลงจากรูป เป็น RLE เหมือนเดิม

In [ ]:
# # Generate rle encodings (images are first converted to the original size)
def resize_mask2rle(p):
    im = PIL.Image.fromarray((p.T*255).astype(np.uint8)).resize((1024,1024), resample=PIL.Image.NEAREST)
    im = np.asarray(im)
    return mask2rle(im, 1024, 1024)

In [ ]:
### Serial Version
rles = []
for p in progress_bar(preds):
    rles.append(resize_mask2rle(p))

# ### Parallel Version
# import multiprocessing as mp
# # mp.cpu_count()
# p=mp.Pool(4)                    
# rles = p.map(resize_mask2rle, preds)

แปลงเป็น DataFrame ก่อน Save เป็นไฟล์ CSV

In [ ]:
ids = [o.stem for o in dl.items]
sub_df = pd.DataFrame({'ImageId': ids, 'EncodedPixels': rles})
sub_df.loc[sub_df.EncodedPixels=='', 'EncodedPixels'] = '-1'
sub_df.head()

In [ ]:
sub_df.shape

In [ ]:
sub_df.to_csv('submission.csv', index=False)

In [ ]:
! kaggle competitions submit siim-acr-pneumothorax-segmentation -f {'submission.csv'} -m "My submission CE min_scale0.7 Ranger 34"

In [ ]:
# Auto Play Audio file
import IPython.display as ipd
ipd.Audio(url="http://www.w3schools.com/html/horse.ogg",
                              autoplay=True) 



| Name           | Submitted    | Wait time | Execution time | Score   |
|----------------|--------------|-----------|----------------|---------|
| submission.csv | a day ago    | 0 seconds | 1 seconds      | 0.9060  |


เราได้ Public Leaderboard 0.9060, Private Leaderboard 0.8104 อยู่ที่อันดับ 280 จาก 1475 ดังตารางด้านล่าง = Top 20%

| #   	| Team Name      	| Score  	| Last 	|
|-----	|----------------	|--------	|------	|
| 278 	| UWRad          	| 0.8113 	| 2mo  	|
| 279 	| Mitul          	| 0.8105 	| 2mo  	|
| 280 	| diCELLa        	| 0.8096 	| 2mo  	|
| 281 	| lucaskg        	| 0.8094 	| 2mo  	|
| 282 	| Insaf Ashrapov 	| 0.8091 	| 2mo  	|



Top 10 ใน Private Leaderboard คือ อันดับที่ 147 จาก 1475 ทีม Dice = 0.8320 

# สรุป

* เราได้สร้าง AI วิจิฉัยภาวะปอดรั่ว (Pneumothorax) ที่ทำงานได้อย่าง อย่างแม่นยำพอสมควร ด้วยการเทรนโมเดล ด้วย GPU ไม่ถึง 24 ชั่วโมง
* แทนที่จะใช้รูป PNG จาก Dataset ที่คนทำไว้ให้แล้ว เราสามารถแปลงไฟล์เองจากไฟล์ DICOM ต้นฉบับ เลือก Window ข้อมูลที่เราต้องการ เพื่อปรับปรุ่งความแม่นยำให้มากขึ้น
* เราสามารถเพิ่มความซับซ้อนของโมเดล, รูปขนาดใหญ่ขึ้น, Loss Function, Multi-Head และใช้ GPU รุ่นใหม่ขึ้นในการเทรน ให้ได้ผลลัพธ์ที่แม่นยำมากขึ้น รวดเร็วขึ้นไปอีก
* เราสามารถประยุกต์ใช้โมเดลนี้ กับโรคที่คล้าย ๆ กันอีกหลายโรค ที่ต้องวินิจฉัยโดย Radiologist (แพทย์รังสีวิทยา) อ่านฟิล์ม X-Ray

# Credit

* https://www.kaggle.com/c/siim-acr-pneumothorax-segmentation/
* https://siim.org/
* https://www.bualabs.com/archives/2856/ai-diagnose-pneumothorax-read-chest-x-ray-radiograph-image-collapsed-lung-radiologist-triage-patient-classification-segment-medical-image-segmentation-ep-2/
* https://www.bualabs.com/archives/835/image-segmentation-semantic-segmentation-camvid-machine-learning-unet-deep-learning-image-segmentation-ep-1/
* https://en.wikipedia.org/wiki/Pneumothorax
* https://www.kaggle.com/schlerp/getting-to-know-dicom-and-the-data/
* https://www.kaggle.com/jesperdramsch/intro-chest-xray-dicom-viz-u-nets-full-data
* https://www.kaggle.com/jesperdramsch/siim-acr-pneumothorax-segmentation-data
* https://www.kaggle.com/iafoss/hypercolumns-pneumothorax-fastai-0-831-lb
* https://www.kaggle.com/iafoss/siimacr-pneumothorax-segmentation-data-128
* https://www.kaggle.com/iafoss/siimacr-pneumothorax-segmentation-data-256
* https://www.kaggle.com/iafoss/siimacr-pneumothorax-segmentation-data-512
* https://www.kaggle.com/iafoss/siimacr-pneumothorax-segmentation-data-1024
* https://www.kaggle.com/mnpinto/pneumothorax-fastai-u-net
* https://www.kaggle.com/c/rsna-pneumonia-detection-challenge
* https://www.kaggle.com/meaninglesslives/unet-with-efficientnet-encoder-in-keras
* https://pydicom.github.io/pydicom/stable/getting_started.html
* https://docs.python.org/2/library/glob.html
* https://arxiv.org/abs/1505.04597
* https://stackoverflow.com/questions/48750199/google-colaboratory-misleading-information-about-its-gpu-only-5-ram-available
* https://github.com/muellerzr/Practical-Deep-Learning-for-Coders-2.0/blob/master/Computer%20Vision/04_Segmentation.ipynb
* https://kornia.readthedocs.io/en/latest/_modules/kornia/losses/focal.html
* https://arxiv.org/abs/1901.05555